In [19]:
# call libraries to be used
using JuMP
using Ipopt

# define algebraic model
m = Model(solver=IpoptSolver())

# define sets
K = ["CO","H2","CH3OH"]; # set of components

# define variables
@variable(m, μ_in[K]>=0)    # inlet molar flows (kmol/hr)
@variable(m, μ_out[K]>=0)   # outlet molar flows (kmol/hr)
@variable(m, μ_out_tot>=0)  # total outlet flow
@variable(m,0<=ξ<=1)        # extent of reactor [-]
@variable(m,P==150)         # reactor pressure [bar]

# define data
T=300+273.15
Keq=10^(-12.275+4938/T) # reaction equilibrium @ 300 degC 

# Gibbs equilibrium condition
@constraint(m, μ_in["CO"] == 100)
@constraint(m, μ_in["H2"] == 600)
@constraint(m, μ_in["CH3OH"]== 0)
@constraint(m, μ_out["CO"]==μ_in["CO"]-100*ξ)
@constraint(m, μ_out["H2"]==μ_in["H2"]-2*100*ξ)
@constraint(m, μ_out["CH3OH"]==100*ξ)
@constraint(m, μ_out_tot==sum(μ_out[k] for k in K))
@NLconstraint(m, 
(μ_out["CH3OH"]/μ_out_tot)/(μ_out["CO"]*μ_out["H2"]*μ_out["H2"]/(μ_out_tot*μ_out_tot*μ_out_tot))== (P^2)*Keq)


In [20]:
# solve problem
solve(m)

This is Ipopt version 3.12.1, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:       19
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:       10

Total number of variables............................:        8
                     variables with only lower bounds:        7
                variables with lower and upper bounds:        1
                     variables with only upper bounds:        0
Total number of equality constraints.................:        8
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

:Optimal

In [21]:
# display solution
println("P=",getvalue(P))
println("ξ=",getvalue(ξ))
println(getvalue(μ_out))
println("μ_out_tot=",getvalue(μ_out_tot))

P=150.0
ξ=0.7669148048452934
μ_out: 1 dimensions:
[   CO] = 23.30851951547066
[   H2] = 446.6170390309413
[CH3OH] = 76.69148048452935
μ_out_tot=546.6170390309413


In [22]:
print(Keq)

0.00021905121614676553